# Query Time!

Comparemos operaciones parecidas en ambas bases de datos.

## Primero hagamos un setup

In [ ]:
import psycopg2
conn = psycopg2.connect("dbname=dota user=dota password=dota host=psql")
postgres = conn.cursor()

In [ ]:
from pymongo import MongoClient
from pprint import pprint
client = MongoClient('mongodb', 27017)
mongodb = client.dota

## Simple query
Postgres:

In [ ]:
postgres.execute("SELECT * FROM heroes WHERE name = 'Tiny'")
postgres.fetchone()

MongoDB:

In [ ]:
mongodb.heroes.find_one({"name": "Tiny"})

## Query con LIKE
Postgres:

In [ ]:
postgres.execute("SELECT * FROM items WHERE description like '%blade%'")
postgres.fetchone()

MongoDB:

In [ ]:
mongodb.items.find_one({"description": {"$regex": u".*blade.*"}})
## Also possible: db.items.find({“description”: /.*blade.*/}), only on mongo cli

## Query con > (greater than)
Postgres:

In [ ]:
postgres.execute("SELECT * FROM items WHERE cost > 500")
postgres.fetchone()

MongoDB:

In [ ]:
mongodb.items.find_one({"cost": {"$gt": 500}})
# In cli: db.items.find({"cost": {$gt: 500}})

## Count
Postgres:

In [ ]:
postgres.execute("SELECT COUNT(*) FROM heroes")
postgres.fetchone()

MongoDB:

In [ ]:
mongodb.heroes.find().count()
# In cli: db.items.find({"cost": {$gt: 500}})

## Nuevas colecciones/tablas
Postgres:

In [ ]:
postgres.execute("""
CREATE TABLE quests(
   ID           INT PRIMARY KEY NOT NULL,
   name         CHAR(50) NOT NULL,
   description  CHAR(250) NOT NULL,
   exp          INT NOT NULL
)""")
conn.commit()

MongoDB:

In [ ]:
mongodb.quests.insert_one({
    "name": "find sword",
    "description": "Find the hidden sword",
    "exp": 500
    })
mongodb.quests.find_one()

__NOTE:__ Se puede utilizar agregation para queries mas interesantes:
```
db.heroes.aggregate(
   [
      {
         $project: {
            name: 1,
            numberOfSkills: { $size: "$skills" }
         }
      }
   ]
)```

## Insertar datos
Postgres:

In [ ]:
conn.rollback()
postgres.execute("""
INSERT INTO quests (id, name, description, exp) VALUES (1, 'find sword', 'Find the hidden sword', 500)
""")
conn.commit()

MongoDB:

In [ ]:
mongodb.quests.insert_many([
    {
    "name": "find armor",
    "description": "Find the hidden armor",
    "exp": 1000
    },
    {
    "name": "find helmet",
    "description": "Find the hidden helmet",
    "exp": 200
    },
])
mongodb.quests.find_one({"name": "find armor"})

__NOTA:__ Los documentos en MongoDB no tienen restricciones de insercion por primary key (se duplican).

## Joins
Postgres:

In [ ]:
conn.rollback()
postgres.execute("""
SELECT * 
    FROM heroes 
    LEFT JOIN skills on skills.hero_id = heroes.id
    WHERE heroes.name = 'Pugna'
""")
heroes_skills = postgres.fetchall()

for hero in heroes_skills:
    pprint(hero)

MongoDB:

In [ ]:
mongodb.heroes.find_one(
    {
    "name": "Pugna"
    })

__NOTA:__ Se pueden hacer joins en mongo:
```
db.heroes.aggregate([{
    $lookup: {
        from: "items",
        localField: "name",
        foreignField: "name",
        as: "items_with_same_name"
    }
},{
    $unwind: "$items_with_same_name"
}])
```

## Modificar tablas - Anadir columnas
Postgres:

In [ ]:
conn.rollback()
postgres.execute("""
ALTER TABLE heroes ADD COLUMN items varchar(30)
""")
conn.commit()

MongoDB:

In [ ]:
mongodb.heroes.update_one(
	{"name": "Drow_Ranger"}, 
	{"$set": {"ext_id": 1}}
)
mongodb.heroes.find_one({"name": "Drow_Ranger"})

## Relacionar datos nuevos entre tablas/colecciones
Postgres:

In [ ]:
conn.rollback()
postgres.execute("""
CREATE TABLE public.heroes_items
(
  id integer NOT NULL DEFAULT nextval('heroes_items_id_seq'::regclass),
  hero_id integer,
  item_id integer,
  obtained timestamp without time zone,
  CONSTRAINT heroes_items_pkey PRIMARY KEY (id),
  CONSTRAINT heroes_items_hero_id_fkey FOREIGN KEY (hero_id)
      REFERENCES public.heroes (id) MATCH SIMPLE
      ON UPDATE NO ACTION ON DELETE NO ACTION,
  CONSTRAINT heroes_items_item_id_fkey FOREIGN KEY (item_id)
      REFERENCES public.items (id) MATCH SIMPLE
      ON UPDATE NO ACTION ON DELETE NO ACTION
)
""")
conn.commit()

In [ ]:
conn.rollback()
postgres.execute("""
INSERT into heroes_items (hero_id, item_id) VALUES (33,26)  
""")
conn.commit()

In [ ]:
postgres.execute("""
    SELECT * FROM heroes 
    LEFT JOIN heroes_items ON heroes.id = heroes_items.hero_id 
    LEFT JOIN items ON heroes_items.item_id = items.id
    WHERE heroes.id = 23;
""")
postgres.fetchone()

MongoDB:

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"}, 
	{"$push": {"items": mongodb.items.find_one()}}
)
mongodb.heroes.find_one({"name": "Visage"})

## Modificar datos existentes
Postgres:

In [ ]:
conn.rollback()
postgres.execute("""
    UPDATE items SET name = 'mega2 sword' WHERE name = 'Blink_Dagger'
""")
conn.commit()

In [ ]:
postgres.execute("""
    SELECT * FROM heroes 
    LEFT JOIN heroes_items ON heroes.id = heroes_items.hero_id 
    LEFT JOIN items ON heroes_items.item_id = items.id
    WHERE heroes.id = 23
""")
postgres.fetchone()

MongoDB:

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"}, 
	{"$pull": {"items": {"name": "Blades_of_Attack"}}}
)
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

In [ ]:
mongodb.items.update_one(
	{"name": "Blades_of_Attack"}, 
	{"$set": {"name": "MEGA Blades of Attack"}}
)
mongodb.items.find_one({"name": "MEGA Blades of Attack"})

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"}, 
	{"$push": {"items": mongodb.items.find_one({"name": "Blink_Dagger"})}}
)
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

## Borrar Elementos
Postgres:

In [ ]:
conn.rollback()
postgres.execute("""
DELETE FROM items WHERE id = 26
""")
conn.commit()

__NOTA:__ Al fallar, lo ideal seria cambiar el constraint:
```
ALTER TABLE table_y   
    DROP CONSTRAINT constraint_name,   
    ADD CONSTRAINT constraint_name FOREIGN KEY (column_in_table_y)
          REFERENCES table_x (referenced_column_in_table_x) ON DELETE CASCADE;
```

MongoDB:

In [ ]:
mongodb.items.delete_one( {"name": "MEGA Blades of Attack"} )
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"}, 
	{"$pull": {"items": {"name": "MEGA Blades of Attack"}}}
)
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

## Agrupar / GROUP BY / ORDER BY
Postgres:

In [ ]:
conn.rollback()
postgres.execute("""
SELECT type, count(*) FROM heroes GROUP BY type ORDER BY type DESC;
""")
hero_types = postgres.fetchall()

for hero in hero_types:
    pprint(hero)

MongoDB:

In [ ]:
aggregation = mongodb.heroes.aggregate([
    {"$group" : {"_id": "$type", "count": {"$sum": 1}}}
])
for results in aggregation:
    print(results)

## EXTRA! MAP REDUCE!

Se pueden efectuar operaciones de tipo map reduce tanto en mongo como en postgresql, aunque el idioma difiere bastante.  

Veamos Postgres primero:

In [ ]:
conn.rollback()
postgres.execute("""CREATE EXTENSION pg_trgm;""")
postgres.execute("""
    with heroes as (
        select id, name
        from heroes
    )
    select similarity('alche', name), *
        from heroes
    order by 1 desc
""")
results = postgres.fetchall()

for similar_hero in results:
    print(similar_hero)

Para MongoDB mejor utilicemos la consola:

```
db.heroes.mapReduce(
    function(){
        var name = this.name;
        for (var idx = 0; idx < this.skills.length; idx++) {
                           emit(name, this.skills[idx].description);
                       }
    },
    function(key, values) {return values.length},
    {
        query: {},
        out: "total_skills"
    }
    )

db.heroes.mapReduce(
    function(){
        var name = this.name;
        if (this.items == null) {
            emit(name, 0);
            return;
        }
        for (var idx = 0; idx < this.items.length; idx++) {
                           emit(name, this.items[idx].cost);
                       }
    },
    function(key, values) {return Array.sum(values)},
    {
        query: {},
        out: "total_skills"
    }
    )

```

In [ ]:
#.find().sort({"value": -1})